# Order Latency Data

To obtain more realistic backtesting results, accounting for latencies is crucial. Therefore, it's important to collect both feed data and order data with timestamps to measure your order latency. The best approach is to gather your own order latencies. You can collect order latency based on your live trading or by regularly submitting orders at a price that cannot be filled and then canceling them for recording purposes. However, if you don't have access to them or want to establish a target, you will need to artificially generate order latency. You can model this latency based on factors such as feed latency, trade volume, and the number of events. In this guide, we will demonstrate a simple method to generate order latency from feed latency using a multiplier and offset for adjustment.

First, loads the feed data.

In [6]:
import numpy as np

data = np.load(r'D:\gptcv\hftbacktest\Data\btcusdt\btcusdt_20240730.npz')['data']
data

array([(3758096385, 1722346072131000000, 1722346072346042089,  1000. , 5.99205e+02, 0, 0, 0.),
       (3758096385, 1722346072131000000, 1722346072346042089,  5000. , 4.29400e+00, 0, 0, 0.),
       (3758096385, 1722346072131000000, 1722346072346042089, 65909.2, 1.50000e-02, 0, 0, 0.),
       ...,
       (3489660929, 1722383999979000000, 1722383999982455596, 66183.3, 0.00000e+00, 0, 0, 0.),
       (3489660929, 1722383999979000000, 1722383999982455596, 66184.5, 5.76000e-01, 0, 0, 0.),
       (3489660929, 1722383999979000000, 1722383999982455596, 66230. , 4.01000e-01, 0, 0, 0.)],
      dtype=[('ev', '<u8'), ('exch_ts', '<i8'), ('local_ts', '<i8'), ('px', '<f8'), ('qty', '<f8'), ('order_id', '<u8'), ('ival', '<i8'), ('fval', '<f8')])

For easy manipulation, converts it into a DataFrame.

In [7]:
import polars as pl

df = pl.DataFrame(data)
df

ev,exch_ts,local_ts,px,qty,order_id,ival,fval
u64,i64,i64,f64,f64,u64,i64,f64
3758096385,1722346072131000000,1722346072346042089,1000.0,599.205,0,0,0.0
3758096385,1722346072131000000,1722346072346042089,5000.0,4.294,0,0,0.0
3758096385,1722346072131000000,1722346072346042089,65909.2,0.015,0,0,0.0
3758096385,1722346072131000000,1722346072346042089,66441.8,1.373,0,0,0.0
3758096385,1722346072131000000,1722346072346042089,66517.0,54.508,0,0,0.0
…,…,…,…,…,…,…,…
3489660929,1722383999979000000,1722383999982455596,66171.0,0.18,0,0,0.0
3489660929,1722383999979000000,1722383999982455596,66172.0,0.08,0,0,0.0
3489660929,1722383999979000000,1722383999982455596,66183.3,0.0,0,0,0.0


Selects only the events that have both a valid exchange timestamp and a valid local timestamp to get feed latency.

In [8]:
from hftbacktest import EXCH_EVENT, LOCAL_EVENT

df = df.filter((pl.col('ev') & EXCH_EVENT == EXCH_EVENT) & (pl.col('ev') & LOCAL_EVENT == LOCAL_EVENT))

Reduces the number of rows by resampling to approximately 1-second intervals.

In [9]:
df = df.with_columns(
    pl.col('local_ts').alias('ts')
).group_by_dynamic(
    'ts', every='1000000000i'
).agg(
    pl.col('exch_ts').last(),
    pl.col('local_ts').last()
).drop('ts')

df

exch_ts,local_ts
i64,i64
1722346072286000000,1722346072355324402
1722346073933000000,1722346073955665228
1722346074948000000,1722346074951565434
1722346075996000000,1722346075998305521
1722346076987000000,1722346076992783671
…,…
1722383995955000000,1722383995966269307
1722383996974000000,1722383996977624480
1722383997984000000,1722383997986074613


Converts back to the structured NumPy array.

In [10]:
data = df.to_numpy(structured=True)
data

array([(1722346072286000000, 1722346072355324402),
       (1722346073933000000, 1722346073955665228),
       (1722346074948000000, 1722346074951565434), ...,
       (1722383997984000000, 1722383997986074613),
       (1722383998965000000, 1722383998967968970),
       (1722383999979000000, 1722383999982455596)],
      dtype=[('exch_ts', '<i8'), ('local_ts', '<i8')])

Generates order latency. Order latency consists of two components: the latency until the order request reaches the exchange's matching engine and the latency until the response arrives backto the localy. Order latency is not the same as feed latency and does not need to be proportional to feed latency. However, for simplicity, we model order latency to be proportional to feed latency using a multiplier and offset.

In [11]:
mul_entry = 4
offset_entry = 0

mul_resp = 3
offset_resp = 0

order_latency = np.zeros(len(data), dtype=[('req_ts', 'i8'), ('exch_ts', 'i8'), ('resp_ts', 'i8'), ('_padding', 'i8')])
for i, (exch_ts, local_ts) in enumerate(data):
    feed_latency = local_ts - exch_ts
    order_entry_latency = mul_entry * feed_latency + offset_entry
    order_resp_latency = mul_resp * feed_latency + offset_resp

    req_ts = local_ts
    order_exch_ts = req_ts + order_entry_latency
    resp_ts = order_exch_ts + order_resp_latency
    
    order_latency[i] = (req_ts, order_exch_ts, resp_ts, 0)
    
order_latency

array([(1722346072355324402, 1722346072632622010, 1722346072840595216, 0),
       (1722346073955665228, 1722346074046326140, 1722346074114321824, 0),
       (1722346074951565434, 1722346074965827170, 1722346074976523472, 0),
       ...,
       (1722383997986074613, 1722383997994373065, 1722383998000596904, 0),
       (1722383998967968970, 1722383998979844850, 1722383998988751760, 0),
       (1722383999982455596, 1722383999996277980, 1722384000006644768, 0)],
      dtype=[('req_ts', '<i8'), ('exch_ts', '<i8'), ('resp_ts', '<i8'), ('_padding', '<i8')])

In [12]:
df_order_latency = pl.DataFrame(order_latency)
df_order_latency

req_ts,exch_ts,resp_ts,_padding
i64,i64,i64,i64
1722346072355324402,1722346072632622010,1722346072840595216,0
1722346073955665228,1722346074046326140,1722346074114321824,0
1722346074951565434,1722346074965827170,1722346074976523472,0
1722346075998305521,1722346076007527605,1722346076014444168,0
1722346076992783671,1722346077015918355,1722346077033269368,0
…,…,…,…
1722383995966269307,1722383996011346535,1722383996045154456,0
1722383996977624480,1722383996992122400,1722383997002995840,0
1722383997986074613,1722383997994373065,1722383998000596904,0


Checks if latency has invalid negative values.

In [13]:
order_entry_latency = df_order_latency['exch_ts'] - df_order_latency['req_ts']
order_resp_latency = df_order_latency['resp_ts'] - df_order_latency['exch_ts']

In [14]:
(order_entry_latency <= 0).sum()

0

In [15]:
(order_resp_latency <= 0).sum()

0

Here, we wrap the entire process into a method with `njit` for increased speed.

In [20]:
import numpy as np
from numba import njit
import polars as pl
from hftbacktest import LOCAL_EVENT, EXCH_EVENT

@njit
def generate_order_latency_nb(data, order_latency, mul_entry, offset_entry, mul_resp, offset_resp):
    for i in range(len(data)):
        exch_ts = data[i].exch_ts
        local_ts = data[i].local_ts
        feed_latency = local_ts - exch_ts
        order_entry_latency = mul_entry * feed_latency + offset_entry
        order_resp_latency = mul_resp * feed_latency + offset_resp

        req_ts = local_ts
        order_exch_ts = req_ts + order_entry_latency
        resp_ts = order_exch_ts + order_resp_latency

        order_latency[i].req_ts = req_ts
        order_latency[i].exch_ts = order_exch_ts
        order_latency[i].resp_ts = resp_ts

def generate_order_latency(feed_file, output_file = None, mul_entry = 1, offset_entry = 0, mul_resp = 1, offset_resp = 0):
    data = np.load(feed_file)['data']
    df = pl.DataFrame(data)
    
    df = df.filter(
        (pl.col('ev') & EXCH_EVENT == EXCH_EVENT) & (pl.col('ev') & LOCAL_EVENT == LOCAL_EVENT)
    ).with_columns(
        pl.col('local_ts').alias('ts')
    ).group_by_dynamic(
        'ts', every='1000000000i'
    ).agg(
        pl.col('exch_ts').last(),
        pl.col('local_ts').last()
    ).drop('ts')
    
    data = df.to_numpy(structured=True)

    order_latency = np.zeros(len(data), dtype=[('req_ts', 'i8'), ('exch_ts', 'i8'), ('resp_ts', 'i8'), ('_padding', 'i8')])
    
    #output_file = r'D:\gptcv\hftbacktest\latency\feed_latency_20240730.npz'
    generate_order_latency_nb(data, order_latency, mul_entry, offset_entry, mul_resp, offset_resp)
    print(order_latency)
    if output_file is not None:
        print(order_latency)
        np.savez_compressed(output_file, data=order_latency)

    return order_latency

In [21]:
order_latency = generate_order_latency(r'D:\gptcv\hftbacktest\Data\1000bonkusdt\1000bonkusdt_20240731.npz', output_file=r'D:\gptcv\hftbacktest\latency\feed_latency_20240731.npz', mul_entry=4, mul_resp=3)

[(1722384000071704170, 1722384000230520850, 1722384000349633360, 0)
 (1722384002990559395, 1722384003008796975, 1722384003022475160, 0)
 (1722384003972886677, 1722384004088433385, 1722384004175093416, 0) ...
 (1722470397951994263, 1722470397959971315, 1722470397965954104, 0)
 (1722470398930866009, 1722470399110330045, 1722470399244928072, 0)
 (1722470399955162711, 1722470399971813555, 1722470399984301688, 0)]
[(1722384000071704170, 1722384000230520850, 1722384000349633360, 0)
 (1722384002990559395, 1722384003008796975, 1722384003022475160, 0)
 (1722384003972886677, 1722384004088433385, 1722384004175093416, 0) ...
 (1722470397951994263, 1722470397959971315, 1722470397965954104, 0)
 (1722470398930866009, 1722470399110330045, 1722470399244928072, 0)
 (1722470399955162711, 1722470399971813555, 1722470399984301688, 0)]
